# The Cat Expert Bot

Install the nesesary stuff

In [ ]:
%pip install pytube
%pip install --upgrade pytube
%pip install yt-dlp
%pip install moviepy
%pip install whisper
%pip install chromadb sentence-transformers
%pip install git+https://github.com/openai/whisper.git
%pip install pytubefix
%pip install chromadb
%pip install langchain
%pip install openai


# LangSmith - LangChain evaluation

In [13]:

from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="cat-expert-knowledge"
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

Load the videos to learn.
Video to Audio

In [14]:
from pytubefix import YouTube

# Replace with your list of YouTube URLs
video_urls = ["https://youtu.be/ZUcVUFvmDFE?si=z9GfOAWF1qothiKs", 
            "https://youtu.be/4DlJYcfiRu4?si=cUVT9L5dEEdkcSt_", 
            "https://youtu.be/rxInrRQLEmM?si=Ai7wHN0dI--cns0x", 
            "https://youtu.be/gxlNfh5ukMw?si=naO3n4VZeXx3PlOs", 
            "https://youtu.be/ojS7XwtoXtw?si=NpNSef7dCm_LnFPv", 
            "https://youtu.be/tsYT7yIOdqQ?si=hdGEpxlmFNMf7NNQ", 
            "https://youtu.be/tsYT7yIOdqQ?si=e_Zdh2dGpqempHR8", 
            "https://youtu.be/UWohxDOXsl4?si=y1nXlUZYw6uzkc8n", 
            "https://youtu.be/gZrwcoiy_gY?si=ksfYE03t6xtuxUL0", 
            "https://youtu.be/lSDI5diNu4Y?si=Q-In6zMD4ZpuaPIz", 
            "https://youtu.be/8aCGL9GpVUg?si=_0yF1U1thjwJqyPY", 
            "https://youtu.be/VjOXvD7OvrE?si=t6xugNxLeMjpsi7E", 
            "https://youtu.be/FzifwTnCV5s?si=sR_u4kG-4NoQx5Ux", 
            "https://youtu.be/XreeFU7RYeI?si=hsc9WO24dJP6AfV2", 
            "https://youtu.be/-4O97jw_8Bc?si=pC14dgZ_f4mXdYPv" ]

for url in video_urls:
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    video.download(output_path="path/to/m4a")


# Transcriptions done by Whisper

In [ ]:
%pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
import os

# Load Whisper model
model = whisper.load_model("base")

# Directory containing your .m4a files
audio_dir = "/notebooks/path/to/m4a/"
transcription_dir = "/notebooks/path/to/transcriptions/"

# Ensure the transcription directory exists
os.makedirs(transcription_dir, exist_ok=True)

# List all .m4a files in the audio directory
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".m4a")]

# Transcribe each audio file and save it as a .txt file
for audio_file in audio_files:
    audio_path = os.path.join(audio_dir, audio_file)
    try:
        transcription = model.transcribe(audio_path)

        # Save transcription to a .txt file
        
        transcription_file = os.path.join(transcription_dir, audio_file.replace(".m4a", ".txt"))
        with open(transcription_file, "w") as f:
            f.write(transcription['text'])
        print(f"Transcribed and saved: {transcription_file}")
    except Exception as e:
        print(f"Failed to transcribe {audio_file}: {e}")



Transcribed and saved: /notebooks/path/to/transcriptions/When Cat Introductions Get UGLY.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips.txt


# Chromadb as Client and creation of Collection

In [ ]:
%pip install --upgrade huggingface_hub
%pip install --upgrade sentence-transformers



In [29]:
import chromadb
from sentence_transformers import SentenceTransformer
import glob

# Initialize ChromaDB client and create a collection
client = chromadb.Client()

# Check if the collection exists; create if not
if "cat_expert_knowledge" not in [col.name for col in client.list_collections()]:
    collection = client.create_collection("cat_expert_knowledge")
else:
    collection = client.get_collection("cat_expert_knowledge")


# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load and process each transcription file
for idx, transcription_file in enumerate(glob.glob("/notebooks/path/to/transcriptions/*.txt")):
    with open(transcription_file, "r") as f:
        text = f.read()

    # Create an embedding for the transcription
    embedding = model.encode(text).tolist()
    topic = os.path.basename(transcription_file).replace(".txt", "")  # Use file name as topic

    # Add to ChromaDB collection with a unique ID
    collection.add(
        ids=[f"doc_{idx}"],  # Unique ID for each document
        documents=[text],
        metadatas=[{"topic": topic, "source": transcription_file}],
        embeddings=[embedding]
    )

print("Transcriptions successfully stored in ChromaDB!")

Insert of existing embedding ID: doc_0
Add of existing embedding ID: doc_0
Insert of existing embedding ID: doc_1
Add of existing embedding ID: doc_1
Insert of existing embedding ID: doc_2
Add of existing embedding ID: doc_2
Insert of existing embedding ID: doc_3
Add of existing embedding ID: doc_3
Insert of existing embedding ID: doc_4
Add of existing embedding ID: doc_4
Insert of existing embedding ID: doc_5
Add of existing embedding ID: doc_5
Insert of existing embedding ID: doc_6
Add of existing embedding ID: doc_6
Insert of existing embedding ID: doc_7
Add of existing embedding ID: doc_7
Insert of existing embedding ID: doc_8
Add of existing embedding ID: doc_8
Insert of existing embedding ID: doc_9
Add of existing embedding ID: doc_9
Insert of existing embedding ID: doc_10
Add of existing embedding ID: doc_10
Insert of existing embedding ID: doc_11
Add of existing embedding ID: doc_11
Insert of existing embedding ID: doc_12
Add of existing embedding ID: doc_12
Insert of existing 

Transcriptions successfully stored in ChromaDB!


# Querying Chromadb

In [30]:
# Sample query to retrieve relevant information
query_text = "How do I stop my cat from scratching furniture?"
query_embedding = model.encode(query_text).tolist()

# Perform the search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

# Check if results are found
if results['documents']:
    for result in results['documents']:
        print("Retrieved Text:", result)
    for metadata in results['metadatas']:
        print("Metadata:", metadata)
else:
    print("No relevant results found.")



Retrieved Text: [" What's up gang is I Jackson Galaxy your cat daddy and today your cat question answerer that is what I'm here for today hopefully we get questions all the time from folks just like you who sent it into this address right there which you of course can do as well and today we're gonna talk to Kinesha Kinesha what's it all about hi Jackson I'm Kinesha from the Cayman Islands my husband and I have a cat sunshine she's 11 years old and we've always wanted to get another cat to you know keep her company and also rescue from the human society but you know we're a bit worried because she has been the only cat in the home for so long in 11 years and we just wonder whether or not she'll be very receptive to the addition of another cat in the home we'd love your feedback on the best approach to do so you know what can we do to make this a seamless process because we'd really love for it to be a success and to avoid any elimination or scratching or you know any of the behaviors t

In [19]:
#%pip install langchain
#%pip install openai
#%pip install langchain_community langchain_openai

In [20]:
import langchain
langchain.__version__

'0.3.7'

In [23]:
from transformers import GPT2Tokenizer
from langchain.chat_models import ChatOpenAI  # Ensure ChatOpenAI is imported correctly
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from sentence_transformers import SentenceTransformer
import chromadb
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence

# Load environment variables from the .env file
load_dotenv("apikey.env")

# Access the API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY is missing. Please set it in your .env file.")

# Initialize ChatOpenAI model with the API key for chat-based models
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

# Initialize ChromaDB client and retrieve the collection
client = chromadb.Client()
collection = client.get_collection("cat_expert_knowledge")

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize a tokenizer to estimate token counts
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Limit context length by token count (e.g., 1500 tokens for context)
MAX_CONTEXT_TOKENS = 1500

# Multi-query generation: create multiple query variations
def generate_query_variations(query):
    """Create variations of a query to enhance retrieval diversity."""
    return [
        query,
        f"Explain about {query}",
        f"How does {query} work?",
        f"What are the details of {query}?",
        f"Tell me more about {query}"
    ]

# Updated retrieve_documents function with token count management
def retrieve_documents(query):
    query_variations = generate_query_variations(query)
    results = []

    for variation in query_variations:
        query_embedding = embedding_model.encode(variation).tolist()
        result = collection.query(query_embeddings=[query_embedding], n_results=2)
        documents = [doc if isinstance(doc, str) else " ".join(doc) for doc in result['documents']]
        results.extend(documents)

    # Combine all retrieved documents
    full_context = " ".join(results)

    # Tokenize and truncate if the context exceeds the maximum token limit
    tokens = tokenizer.encode(full_context)
    if len(tokens) > MAX_CONTEXT_TOKENS:
        tokens = tokens[:MAX_CONTEXT_TOKENS]
        truncated_context = tokenizer.decode(tokens)
    else:
        truncated_context = full_context

    return truncated_context

# Define the LangChain ChatPromptTemplate with messages for chat models
def create_qa_chain():
    # Chat prompt template for structured input to chat model
    template = """You are a friendly, expert cat behaviorist inspired by Jackson Galaxy.
Use the information below to answer the question comprehensively."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{context}\n\nQuestion: {question}\nAnswer:")
    ])

    # Define the LLMChain with the chat prompt for generating answers
    #return LLMChain(llm=llm, prompt=prompt)
    return RunnableSequence(prompt | llm)

# Main agent to respond to user queries
context_history = []

def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")
    
    # Retrieve relevant context from ChromaDB based on query variations
    conversation_context = retrieve_documents(query)
    
    # Create QA chain with the generated context
    qa_chain = create_qa_chain()
    #answer = qa_chain.run({"context": conversation_context, "question": query})
    answer = qa_chain.invoke({"context": conversation_context, "question": query})


    # Append the answer to the context history for continuity
    context_history.append(f"Answer: {answer}")
    return answer

# Test the agent with a sample query
user_query = "Why does my cat scratch furniture?"
response = ask_agent(user_query)
print("Agent Response:", response)


Token indices sequence length is longer than the specified maximum sequence length for this model (24579 > 1024). Running this sequence through the model will result in indexing errors


Agent Response: content="There are several reasons why cats scratch furniture:\n\n1. **Territory Marking**: Cats have scent glands in their paws, and scratching is a way for them to mark their territory. By scratching furniture, they are leaving both a visual mark (the scratches) and a scent mark (from the glands in their paws).\n\n2. **Stretching and Exercise**: Scratching helps cats stretch their muscles, particularly in their shoulders and back. It also allows them to exercise and maintain their claws.\n\n3. **Stress Relief**: Scratching can be a way for cats to relieve stress or anxiety. It helps them release pent-up energy and express their emotions.\n\n4. **Maintaining Claws**: Scratching helps cats remove the outer sheath of their claws, keeping them healthy and sharp.\n\nTo prevent your cat from scratching furniture, you can try the following tips:\n\n1. **Provide Scratching Posts**: Offer your cat appropriate scratching surfaces like scratching posts or pads. Place them near t

In [24]:
def create_qa_chain():
    # Chat prompt template for structured input to chat model
    template = """You are a friendly, expert cat behaviorist inspired by Jackson Galaxy.
Use the information below to provide insightful and supportive advice as if speaking to a new cat owner.

{context}

Question: {question}
Answer in a friendly and approachable manner:"""

    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{context}\n\nQuestion: {question}\nAnswer:")
    ])

    # Define the LLMChain with the chat prompt for generating answers
    return LLMChain(llm=llm, prompt=prompt)


In [25]:
# Main agent to respond to user queries with conversational memory
def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")

    # Retrieve relevant context from ChromaDB based on query variations
    conversation_context = retrieve_documents(query)
    
    # Combine recent exchanges for conversational memory
    recent_history = "\n".join(context_history[-3:])  # Keep last 3 exchanges for continuity
    combined_context = recent_history + "\n\n" + conversation_context
    
    # Create QA chain with the combined context
    qa_chain = create_qa_chain()
    answer = qa_chain.run({"context": combined_context, "question": query})

    # Append the answer to the context history for continuity
    context_history.append(f"Answer: {answer}")
    return answer


# LangSmith Check Point.

This is only for review of how its working

In [26]:
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.stdout import StdOutCallbackHandler


# Initialize Tracer callback handler for LangChain's built-in tracing
tracer_handler = StdOutCallbackHandler()

# Attach the callback handler to a CallbackManager
callback_manager = CallbackManager([tracer_handler])

# Main agent function with LangChain tracing
def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")

    # Retrieve relevant context from ChromaDB based on query variations
    conversation_context = retrieve_documents(query)
    
    # Combine recent exchanges for conversational memory
    recent_history = "\n".join(context_history[-3:])  # Keep last 3 exchanges for continuity
    combined_context = recent_history + "\n\n" + conversation_context
    
    # Create QA chain with the combined context
    qa_chain = create_qa_chain()

    # Use callback_manager to log the QA chain run
    answer = qa_chain.run(
        {"context": combined_context, "question": query},
        callbacks=callback_manager
    )

    # Append the answer to the context history for continuity
    context_history.append(f"Answer: {answer}")
    return answer


In [ ]:
#%pip install yt-dlp

# Download Videos

In [27]:
import yt_dlp
import os

# Define the playlist URL and output path for saving video files
playlist_url = "https://www.youtube.com/playlist?list=PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc"  # Replace with actual playlist URL
output_path = "path/to/save/videos/%(title)s.%(ext)s"  # Directory to save videos

# Set yt-dlp options for downloading video
ydl_opts = {
    'format': 'bestvideo+bestaudio',  # Downloads the best quality video and audio
    'outtmpl': output_path,           # Sets the output path and filename format
    'ignoreerrors': True,             # Skip any videos that cause errors
}

# Download videos from the specified playlist
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([playlist_url])

print("Videos have been successfully downloaded.")


[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc
[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc: Downloading webpage
[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: INTRODUCTIONS
[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] PLAJvHNBwbBNvSQFSeeHswdgIk2m4fljOc page 1: Downloading API JSON


[youtube:tab] Playlist INTRODUCTIONS: Downloading 15 items of 15
[download] Downloading item 1 of 15
[youtube] Extracting URL: https://www.youtube.com/watch?v=ZUcVUFvmDFE
[youtube] ZUcVUFvmDFE: Downloading webpage
[youtube] ZUcVUFvmDFE: Downloading ios player API JSON
[youtube] ZUcVUFvmDFE: Downloading mweb player API JSON
[youtube] ZUcVUFvmDFE: Downloading m3u8 information
[info] ZUcVUFvmDFE: Downloading 1 format(s): 616+251
[download] path/to/save/videos/When Cat Introductions Get UGLY.webm has already been downloaded
[download] Downloading item 2 of 15
[youtube] Extracting URL: https://www.youtube.com/watch?v=4DlJYcfiRu4
[youtube] 4DlJYcfiRu4: Downloading webpage
[youtube] 4DlJYcfiRu4: Downloading ios player API JSON
[youtube] 4DlJYcfiRu4: Downloading mweb player API JSON
[youtube] 4DlJYcfiRu4: Downloading m3u8 information
[info] 4DlJYcfiRu4: Downloading 1 format(s): 616+251
[download] path/to/save/videos/How To Introduce Your Cat to a New KITTEN!.webm has already been downloaded
[d

In [ ]:
from langsmith import LangSmithCallbackHandler
from langchain.callbacks import CallbackManager

# Initialize LangSmith callback handler
langsmith_handler = LangSmithCallbackHandler(
    project_name="Cat Behavior Chatbot",
    tags=["production", "chatbot"]
)

# Attach the callback handler to a CallbackManager
callback_manager = CallbackManager([langsmith_handler])

# Update `ask_agent` to use callback for logging
def ask_agent(query, user_id="default_user"):
    # Get user profile and preferred response length
    profile = get_user_profile(user_id)
    
    # Detect intent and set response type accordingly
    intent = detect_intent(query)
    response_length = profile["preferred_response_length"] if intent != "concise" else "concise"
    
    # Manage memory based on user preferences and recent interactions
    conversation_context = manage_memory(query, user_id)
    
    # Retrieve relevant context and images if intent is "visual_example"
    if intent == "visual_example":
        relevant_texts, relevant_images = retrieve_documents(query)
    else:
        relevant_texts, _ = retrieve_documents(query)
        relevant_images = []

    # Combine text responses
    text_response = " ".join([text["metadata"]["text"] for text in relevant_texts])

    # Create a customized response based on intent and user preferences
    response = f"**{profile['name']}'s Personalized Advice:**\n{text_response}"
    if intent == "visual_example" and relevant_images:
        response += "\n\n**Visual References:**\n" + "\n".join(relevant_images)

    # Append answer to user-specific history
    get_user_history(user_id).append(f"Question: {query}\nAnswer: {response}")
    
    # Run with callbacks for LangSmith logging
    callback_manager.on_text(query)
    callback_manager.on_text(response)

    return response


In [ ]:
%pip install opencv-python

# Download and Extract Key Frames from Videos

In [28]:
import cv2
import os

# Directory containing videos
video_dir = "/notebooks/path/to/save/videos"
frame_dir = "path/to/frames"

os.makedirs(frame_dir, exist_ok=True)

def extract_key_frames(video_path, output_folder, intervals=60):
    """Extract frames at given intervals (in seconds) from a video."""
    video_name = os.path.basename(video_path).split('.')[0]
    capture = cv2.VideoCapture(video_path)
    fps = capture.get(cv2.CAP_PROP_FPS)
    interval_frames = int(fps * intervals)

    success, frame = capture.read()
    count = 0
    while success:
        if count % interval_frames == 0:
            frame_name = f"{video_name}_frame_{count}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_name), frame)
        success, frame = capture.read()
        count += 1
    capture.release()

# Extract frames from each video
for video_file in os.listdir(video_dir):
    extract_key_frames(os.path.join(video_dir, video_file), frame_dir)


KeyboardInterrupt: 

# Embed and Store Visualy with Clip

In [ ]:
%pip install transformers
%pip install langsmith
%pip install --upgrade --quiet  langchain-experimental
%pip install --upgrade --quiet  pillow open_clip_torch torch matplotlib

In [ ]:
import open_clip

open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_image_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_text_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_basic_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_s13m_b4k'),
 ('ViT-

In [ ]:
model_name = "ViT-g-14"
checkpoint = "laion2b_s34b_b88k"

In [ ]:
import numpy as np
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from PIL import Image

# Initialize the OpenCLIP embedding model
clip_embd = OpenCLIPEmbeddings(model_name="ViT-g-14", checkpoint="laion2b_s34b_b88k")


open_clip_pytorch_model.bin:   0%|          | 0.00/5.47G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Generate Embeddings for Text and Images

In [ ]:
# Sample text documents
text_documents = [
    {"id": "1", "text": "How to reduce cat scratching behavior", "topic": "behavior"},
    {"id": "2", "text": "Creating a cat-friendly environment", "topic": "environment"}
]

# Embed and store text embeddings
text_embeddings = []
for doc in text_documents:
    embedding = clip_embd.embed_documents([doc['text']])
    text_embeddings.append({
        "id": doc["id"],
        "embedding": embedding,
        "metadata": {"topic": doc["topic"], "text": doc["text"]}
    })


In [ ]:
import os

# Set the directory for frames
frame_dir = "/notebooks/path/to/frames"
os.makedirs(frame_dir, exist_ok=True)  # Ensure the directory exists

# Example for embedding and storing image embeddings
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from PIL import Image

# Initialize OpenCLIP embeddings
clip_embd = OpenCLIPEmbeddings(model_name="ViT-g-14", checkpoint="laion2b_s34b_b88k")

# Embed and store image embeddings
image_embeddings = []
for frame_file in os.listdir(frame_dir):
    frame_path = os.path.join(frame_dir, frame_file)
    image = Image.open(frame_path)
    embedding = clip_embd.embed_image([frame_path])  # Embed using the new path
    image_embeddings.append({
        "id": frame_file,
        "embedding": embedding,
        "metadata": {"topic": "visual", "source": frame_path}
    })


In [ ]:
# Image file paths
#image_files = ["/notebooks/path/to/frames/Non-Recognition Aggression in Cats Explained_frame_0.jpg", 
 #              "/notebooks/path/to/frames/Non-Recognition Aggression in Cats Explained_frame_1438.jpg"]

# Embed and store image embeddings
#image_embeddings = []
#for img_path in image_files:
#    embedding = clip_embd.embed_image([img_path])
#    image_embeddings.append({
#        "id": img_path,
#        "embedding": embedding,
#        "metadata": {"topic": "visual", "source": img_path}
#    })

Store Embeddings and Perform Retrieval:
Since LangSmith doesn’t yet support a full embedding storage and retrieval service like ChromaDB, we’ll use numpy to store and retrieve embeddings locally. This will simulate a vector search.

In [ ]:
# Store embeddings in separate lists for text and images
text_store = {entry["id"]: entry for entry in text_embeddings}
image_store = {entry["id"]: entry for entry in image_embeddings}


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_similar_embeddings(query, store, top_k=2):
    """Retrieve top K similar embeddings from the store."""
    query_embedding = clip_embd.embed_documents([query])
    similarities = []

    for item in store.values():
        score = cosine_similarity(query_embedding, item["embedding"]).flatten()[0]
        similarities.append((score, item))

    # Sort by similarity and select top K results
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
    return [item for _, item in similarities[:top_k]]


# Update LangChain Agent for Multimodal Retrieval

In [ ]:
def ask_agent(query):
    # Retrieve similar text and image embeddings
    relevant_texts = retrieve_similar_embeddings(query, text_store)
    relevant_images = retrieve_similar_embeddings(query, image_store)
    
    # Combine text responses
    text_response = " ".join([text["metadata"]["text"] for text in relevant_texts])
    
    # Prepare image references for output
    image_references = "\n".join([image["metadata"]["source"] for image in relevant_images])

    # Final response with text and image references
    response = f"{text_response}\n\nVisual references available:\n{image_references}"
    return response

# Test the agent with a sample query
user_query = "How can I create a cat-friendly environment?"
response = ask_agent(user_query)
print("Agent Response:", response)


Agent Response: Creating a cat-friendly environment How to reduce cat scratching behavior

Visual references available:



In [ ]:
def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")

    # Retrieve relevant context and images from ChromaDB based on query variations
    conversation_context, images = retrieve_documents(query)
    
    # Combine recent exchanges for conversational memory
    recent_history = "\n".join(context_history[-3:])
    combined_context = recent_history + "\n\n" + conversation_context
    
    # Create QA chain with the combined context
    qa_chain = create_qa_chain()
    answer = qa_chain.run({"context": combined_context, "question": query})

    # Append the answer to the context history for continuity
    context_history.append(f"Answer: {answer}")
    
    # Present images in the response if available
    if images:
        answer += "\nVisual references available:\n" + "\n".join(images)

    return answer


# Optimize Multimodal Retrieval and Response Formatting

In [ ]:
def ask_agent(query):
    # Retrieve similar text and image embeddings
    relevant_texts = retrieve_similar_embeddings(query, text_store)
    relevant_images = retrieve_similar_embeddings(query, image_store)
    
    # Combine text responses
    text_response = " ".join([text["metadata"]["text"] for text in relevant_texts])
    
    # Prepare image references for output
    image_references = "\n".join([image["metadata"]["source"] for image in relevant_images])

    # Final response with formatted sections
    response = (
        f"**Expert Advice:**\n{text_response}\n\n"
        f"**Visual References:**\n{image_references if image_references else 'No visual references available.'}"
    )
    return response

# Test the agent with a sample query
user_query = "How can I create a cat-friendly environment?"
response = ask_agent(user_query)
print("Agent Response:", response)


Agent Response: **Expert Advice:**
Creating a cat-friendly environment How to reduce cat scratching behavior

**Visual References:**
No visual references available.


# Implement a Feedback Loop

In [ ]:
import json

# Define a function to log feedback
feedback_log = "feedback_log.json"  # Path to store feedback

def log_feedback(query, response, rating, comment=None):
    feedback_entry = {
        "query": query,
        "response": response,
        "rating": rating,
        "comment": comment
    }
    
    # Append feedback to a JSON file
    try:
        with open(feedback_log, "a") as f:
            f.write(json.dumps(feedback_entry) + "\n")
    except IOError:
        print("Error logging feedback.")

# Collect feedback from the user
def collect_feedback(query, response):
    print("Rate the helpfulness of the response (1-5):")
    rating = int(input("Rating: "))
    comment = input("Any comments? ")
    
    # Log the feedback
    log_feedback(query, response, rating, comment)
    print("Thank you for your feedback!")

# Test the agent with feedback
user_query = "Why does my cat scratch furniture?"
response = ask_agent(user_query)
print("Agent Response:", response)

# Collect feedback from user
collect_feedback(user_query, response)


Agent Response: **Expert Advice:**
How to reduce cat scratching behavior Creating a cat-friendly environment

**Visual References:**
No visual references available.
Rate the helpfulness of the response (1-5):
Thank you for your feedback!


# Implement Personalization

In [ ]:
# Dictionary to store user profiles
user_profiles = {}

# Function to initialize or update a user profile
def get_user_profile(user_id):
    if user_id not in user_profiles:
        user_profiles[user_id] = {
            "name": "User",
            "preferred_response_length": "detailed",  # or 'concise'
            "interest_areas": ["behavior", "environment"]
        }
    return user_profiles[user_id]

# Function to update profile details
def update_user_profile(user_id, name=None, response_length=None, interests=None):
    profile = get_user_profile(user_id)
    if name:
        profile["name"] = name
    if response_length:
        profile["preferred_response_length"] = response_length
    if interests:
        profile["interest_areas"] = interests


# Optimize Memory Management

In [ ]:
# Limit memory length to relevant recent exchanges
def manage_memory(query, user_id, max_exchanges=3):
    profile = get_user_profile(user_id)
    relevant_context = "\n".join([context for context in context_history[-max_exchanges:] 
                                  if any(area in context for area in profile["interest_areas"])])
    return relevant_context


# Intent Handling

We’ll add a simple intent classification to help the chatbot recognize and respond to different types of user intents. Basic intents might include requests for:

Advice: General guidance or recommendations.
Visual Examples: Requests for images or visual aids.
Detailed Explanations: In-depth responses for complex questions.
Define Intent Detection
For simplicity, we’ll use keywords to detect intents. For more robust intent classification, you could use a trained NLP model.

In [ ]:
# Detect user intent based on keywords
def detect_intent(query):
    query = query.lower()
    if any(word in query for word in ["how", "what", "why"]):
        return "advice"
    elif any(word in query for word in ["show", "picture", "visual", "image"]):
        return "visual_example"
    elif any(word in query for word in ["explain", "detailed", "in-depth"]):
        return "detailed_explanation"
    else:
        return "advice"  # Default intent


# Modify ask_agent to Use Personalization, Memory Management, and Intent Handling
Update the main function to:

Retrieve the user’s profile.
Use the detected intent to adjust the response.
Apply memory management to provide relevant context.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Mock storage for embeddings (use actual embeddings in a real setup)
text_store = {}
image_store = {}

# Define `retrieve_documents` function
def retrieve_documents(query, top_k=2):
    # Embed the query using the same embedding function used for storing text and images
    query_embedding = clip_embd.embed_documents([query])
    
    # Retrieve text embeddings
    text_results = []
    for item in text_store.values():
        score = cosine_similarity(query_embedding, item["embedding"]).flatten()[0]
        text_results.append((score, item))
    text_results = sorted(text_results, key=lambda x: x[0], reverse=True)[:top_k]
    
    # Retrieve image embeddings
    image_results = []
    for item in image_store.values():
        score = cosine_similarity(query_embedding, item["embedding"]).flatten()[0]
        image_results.append((score, item))
    image_results = sorted(image_results, key=lambda x: x[0], reverse=True)[:top_k]
    
    # Format the results
    relevant_texts = [item[1] for item in text_results]
    relevant_images = [item[1]["metadata"]["source"] for item in image_results]
    
    return relevant_texts, relevant_images


In [ ]:
%pip install --upgrade gradio
%pip install SpeechRecognition
%pip install gTTS




# Gradio deployment

In [ ]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import tempfile
# Ensure `ask_agent` is imported from the correct location

# Convert audio to text using Google Speech Recognition
def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Sorry, I could not understand the audio."
        except sr.RequestError:
            return "Could not request results; check your internet connection."

# Convert text to speech using gTTS
def text_to_speech_g


import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import tempfile
from main import ask_agent  # Import the ask_agent function

# Convert audio to text using Google Speech Recognition
def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Sorry, I could not understand the audio."
        except sr.RequestError:
            return "Could not request results; check your internet connection."

# Convert text to speech using gTTS
def text_to_speech_gtts(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        return fp.name

# Handle user input for Gradio interface
def handle_input(input_text=None, audio_file=None):
    # Transcribe audio if provided, otherwise use text input
    if audio_file is not None:
        transcribed_text = audio_to_text(audio_file)
        if transcribed_text.startswith("Sorry"):
            return transcribed_text, None  # Handle failed transcription
    else:
        transcribed_text = input_text  # Use provided text input if no audio

    # Process the input using ask_agent
    response_text = ask_agent(transcribed_text, user_id="default_user")
    
    # Convert response to audio
    audio_response = text_to_speech_gtts(response_text)
    return response_text, audio_response

# Create Gradio interface
iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(placeholder="Type your question here...", label="Text Input"),
        gr.Audio(type="filepath", label="Or speak your question")
    ],
    outputs=[
        gr.Textbox(label="Cat Expert's Response"),
        gr.Audio(label="Spoken Response")
    ],
    title="Cat Behavior Expert Chatbot",
    description="Ask questions about cat behavior, either by typing or speaking.",
)

# Launch the interface
iface.launch(share=True)
